# Project Title: Property Usage
## Data cleansing for tax parcels
## By Go Code CO team "Tech Slope" - Aaron, Adam, Dex, Leo, Marc
### Version 0.7
### Last update: May 13, 2019

In [1]:
### Initiate
!pip install geopandas

import pandas as pd
import geopandas as gpd
import time
import numpy as np
import matplotlib as plt
from matplotlib.path import Path
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Task 1 Load and Clean Denver Parcels

In [2]:
### Load parcles
workdir='gdrive/My Drive/GoCode - PropertyUsage/Data processing/'
filename=workdir+'Raw Data/denver_parcels.csv'
df_den = pd.read_csv(filename,na_values='None')
df_den.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,11,12,13,14,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PIN,SCHEDNUM,MAPNUM,BLKNUM,PARCELNUM,APPENDAGE,PARCEL_SOURCE,SYSTEM_START_DATE,OWNER_NAME,OWNER_ADDRESS_LINE1,OWNER_ADDRESS_LINE2,OWNER_ADDR_NBR_PREFIX,OWNER_ADDR_NBR,OWNER_ADDR_NBR_SUFFIX,OWNER_STR_NAME_PRE_MOD,OWNER_STR_NAME_PRE_DIR,OWNER_STR_NAME_PRE_TYPE,OWNER_STR_NAME,OWNER_STR_NAME_POST_TYPE,OWNER_STR_NAME_POST_DIR,OWNER_STR_NAME_POST_MOD,OWNER_UNIT_TYPE,OWNER_UNIT_IDENT,OWNER_CITY,OWNER_STATE,OWNER_ZIP,SITUS_ADDRESS_ID,SITUS_ADDRESS_LINE1,SITUS_ADDRESS_LINE2,SITUS_CITY,SITUS_STATE,SITUS_ZIP,SITUS_ADDR_NBR,SITUS_ADDR_NBR_SUFFIX,SITUS_STR_NAME_PRE_MOD,SITUS_STR_NAME_PRE_DIR,SITUS_STR_NAME_PRE_TYPE,SITUS_STR_NAME,SITUS_STR_NAME_POST_TYPE,SITUS_STR_NAME_POST_DIR,SITUS_STR_NAME_POST_MOD,SITUS_UNIT_TYPE,SITUS_UNIT_IDENT,TAX_DIST,LAND_VALUE,IMPROVEMENT_VALUE,IMPROVEMENTS,PROP_CLASS,TOTAL_VALUE,LAND,D_CLASS,D_CLASS_CN,DCL12,ACT_ZONE,IMP_AREA,CCYRBLT,ASAL_INSTR,SALE_MONTHDAY,SALE_YEAR,SALE_PRICE,RECEPTION_NUM
0,160820474.0,226543004000,2265.0,43.0,4,0,,2010-12-09 00:00:00,"REYNOLDS,TRAVIS W",2634 N LAFAYETTE ST,NaN,NaN,2634,NaN,NaN,N,NaN,LAFAYETTE,ST,NaN,NaN,NaN,NaN,DENVER,CO,80205-4447,41610.0,2634 N LAFAYETTE ST,NaN,DENVER,CO,80205-4447,2634,NaN,NaN,N,NaN,LAFAYETTE,ST,NaN,NaN,NaN,NaN,DENVER,269000.0,422700.0,30434.0,1112.0,691700.0,4690.0,117,SINGLE FAMILY,11.0,U-SU-B1,2032.0,1884.0,WD,401.0,2005.0,347000.0,2005062391
1,160785474.0,225247015000,2252.0,47.0,15,0,Sanborn,2010-12-09 00:00:00,"CRIBBEN,HILLARY HAMLIM",3237 N JOSEPHINE ST,NaN,NaN,3237,NaN,NaN,N,NaN,JOSEPHINE,ST,NaN,NaN,NaN,NaN,DENVER,CO,80205-4154,38077.0,3237 N JOSEPHINE ST,NaN,DENVER,CO,80205-4154,3237,NaN,NaN,N,NaN,JOSEPHINE,ST,NaN,NaN,NaN,NaN,DENVER,164500.0,298500.0,21492.0,1112.0,463000.0,5000.0,113,SINGLE FAMILY,11.0,U-SU-A1,1253.0,1919.0,QC,1111.0,2005.0,NaN,2005219729
2,160971316.0,235112005000,2351.0,12.0,5,0,,2010-12-09 00:00:00,"LOPEZ,ANDREW",2452 N WILLIAMS ST,NaN,NaN,2452,NaN,NaN,N,NaN,WILLIAMS,ST,NaN,NaN,NaN,NaN,DENVER,CO,80205-5524,55938.0,2452 N WILLIAMS ST,NaN,DENVER,CO,80205-5524,2452,NaN,NaN,N,NaN,WILLIAMS,ST,NaN,NaN,NaN,NaN,DENVER,200200.0,330700.0,23810.0,1112.0,530900.0,3140.0,117,SINGLE FAMILY,11.0,U-SU-B1,1791.0,1906.0,WD,720.0,2011.0,341000.0,2011082643
3,161253047.0,510406013000,5104.0,6.0,13,0,Sanborn,2010-12-09 00:00:00,"TAFFE,GREG",18 N SHERMAN ST,NaN,NaN,18,NaN,NaN,N,NaN,SHERMAN,ST,NaN,NaN,NaN,NaN,DENVER,CO,80203-4002,80865.0,18 N SHERMAN ST,NaN,DENVER,CO,80203-4002,18,NaN,NaN,N,NaN,SHERMAN,ST,NaN,NaN,NaN,NaN,DENVER,224600.0,325900.0,23465.0,1112.0,550500.0,3060.0,117,SINGLE FAMILY,11.0,G-MU-3,1637.0,1901.0,WD,123.0,2012.0,260000.0,2012010498
4,160791768.0,225342016000,2253.0,42.0,16,0,,2010-12-09 00:00:00,"KELLER,MELISSA D",2720 N COLUMBINE ST,NaN,NaN,2720,NaN,NaN,N,NaN,COLUMBINE,ST,NaN,NaN,NaN,NaN,DENVER,CO,80205-4742,38712.0,2720 N COLUMBINE ST,NaN,DENVER,CO,80205-4742,2720,NaN,NaN,N,NaN,COLUMBINE,ST,NaN,NaN,NaN,NaN,DENVER,196400.0,262400.0,18893.0,1112.0,458800.0,6650.0,113,SINGLE FAMILY,11.0,U-SU-B1,1378.0,1921.0,WD,531.0,2007.0,150000.0,2007102201


In [3]:
def nancheck(df):
    print('Shape of the dataframe:',df.shape,'\n')
    print('Missing value counts:')
    print(df.shape[0]-df.count(),'\n') 
    return  
# Saving the new file/checkpoint
def write_checkpoint(df, dfname):
  filename=time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename)
  
## Cleansing
cols_den=['SITUS_ADDRESS_LINE1','SITUS_CITY','LAND','D_CLASS_CN','ACT_ZONE','IMP_AREA']
df_den_clean=df_den[cols_den]
df_den_clean['Area']=df_den_clean[["LAND", "IMP_AREA"]].max(axis=1)
df_den_clean.drop(columns=["LAND", "IMP_AREA"],inplace=True)
df_den_clean=df_den_clean.rename(index=str, columns={'SITUS_ADDRESS_LINE1':'Address','SITUS_CITY':'City',"SITUS_ZIP":'ZIP','MAPNUM':'LotNum','BLKNUM':'BlockNum','D_CLASS_CN':'PropertySubtype','ACT_ZONE':'Zoning'})
nancheck(df_den_clean) ## 1303 out of 231668 records has no zoning info
df_den_clean=df_den_clean.dropna(axis=0,how='any')
df_den_clean['County']='DENVER'
df_den_clean['CountyID']='031'
df_den_clean.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Shape of the dataframe: (231668, 5) 

Missing value counts:
Address             122
City                122
PropertySubtype    1368
Zoning             1303
Area               1303
dtype: int64 



,Address,City,PropertySubtype,Zoning,Area,County,CountyID
0,2634 N LAFAYETTE ST,DENVER,SINGLE FAMILY,U-SU-B1,4690.0,DENVER,031
1,3237 N JOSEPHINE ST,DENVER,SINGLE FAMILY,U-SU-A1,5000.0,DENVER,031
2,2452 N WILLIAMS ST,DENVER,SINGLE FAMILY,U-SU-B1,3140.0,DENVER,031
3,18 N SHERMAN ST,DENVER,SINGLE FAMILY,G-MU-3,3060.0,DENVER,031
4,2720 N COLUMBINE ST,DENVER,SINGLE FAMILY,U-SU-B1,6650.0,DENVER,031


In [0]:
## Create list of types for mapping by CRE specialists.
grouped=pd.DataFrame(df_den_clean['PropertySubtype'].unique())
grouped
write_checkpoint(grouped,'grouped')

In [0]:
### Create list of unique zoning codes for mapping by CRE specialists.
grouped=df_clean.groupby('ZoningCode')['PropertySubtype'].apply(np.unique)
grouped.head()

ZoningCode
B-1             [CHURCH BUILT AS CHURCH, CONDOMINIUM, FINANCIA...
B-2             [APT MID-RISE>9 UNT,1-9STY, AUTO SERVICE, CARW...
B-3             [101 CONDO PKG, 10S CONDO STORAGE, APT MID-RIS...
B-4             [APT LOW-RISE>9UNT, WALK-UP, APT MID-RISE>9 UN...
B-5                                        [OTHER REC FACILITIES]
Name: PropertySubtype, dtype: object

In [0]:
def write_checkpoint(df, dfname):
  filename=workdir+time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
write_checkpoint(grouped,'DenverZoningType_grouped')
write_checkpoint(df_den_clean,'DenverParcels_ML')

## df_mesazone = pd.read_csv('Mesa_Zone.csv',dtype={"StreetNumber":str,'OppZoneGeoID':str})
## df_mesazone.head()
nancheck(df_clean)

## Task 2 Load and Clean Mesa Parcels and Zoning Information


In [0]:
filename_mesa=workdir+'Raw Data/Mesa_Parcels/Parcel.shp'
df_mesa = gpd.read_file(filename_mesa)
df_mesa.head()
##df_mesa.area.head()['area'] = data.area

,ACCOUNTNO,PARCELNUM,PARCEL_NUM,ASSOCPAR,LOCATION,SITUS_CITY,SITUS_STAT,MAILING,MAILING_CI,MAILING_ST,...,TOTASSPRE,ZONING,Acres,GlobalID,JURISDICTI,FRPARCEL,FRACCTNO,Shape_STAr,Shape_STLe,geometry
0,R007296,269707312003,2697-073-12-003,None,436 BREEZE DR,FRUITA,CO,1820 K 4/10 RD,FRUITA,CO,...,12030.0,None,0.168585,{ADBD884C-DAE4-4E11-B709-000037E9846D},7,2697-073-00-057,R007275,682.238770,106.553600,"POLYGON ((694540.5321000004 4337525.6522, 6945..."
1,R001645,266132202001,2661-322-02-001,None,67481 BLACK BEAR DR,COLLBRAN,CO,594 MAXWELL DR,GRAND JUNCTION,CO,...,5240.0,AFT,0.325785,{D8688B0C-CC5E-4647-AC5A-0000B3D504B1},0,0000-000-00-000,0,1318.405518,147.118531,"POLYGON ((776771.8103999998 4347909.269099999,..."
2,R071402,294525309002,2945-253-09-002,None,212 27 RD,GRAND JUNCTION,CO,212 27 RD APT B,GRAND JUNCTION,CO,...,10260.0,RSF-4,0.521364,{963C6858-DC61-469F-8706-0001153A90EB},2,0000-000-00-000,0,2109.883789,264.909615,"POLYGON ((711948.5001999997 4323552.819499999,..."
3,M082896,700805305162,7008-053-05-162,2945-262-10-001,651 S HIGHWAY 50 #2,GRAND JUNCTION,CO,2679 B 1/2 RD,GRAND JUNCTION,CO,...,180.0,None,0.014178,{85AE9C97-E213-4D51-BD5D-000247B90E80},1,0000-000-00-000,0,57.374756,43.157331,"POLYGON ((710892.1960000005 4324753.83, 710877..."
4,R017377,270130413001,2701-304-13-001,None,803 MEASE RD,GRAND JUNCTION,CO,267 W DANBURY CIR,GRAND JUNCTION,CO,...,30570.0,RSF-E,2.007886,{A1B49BCA-38D0-43FC-89B3-000248282613},2,2701-304-09-001,R017362,8125.627686,357.862384,"POLYGON ((704556.9907999998 4332946.022299999,..."


In [0]:
## Cleansing
cols_mesa=['LOCATION','SITUS_CITY','ARCH1ST','ZONING']
df_mesa_clean=df_mesa[cols_mesa]
df_mesa_clean['Area']=df_mesa['Acres']*43560
df_mesa_clean=df_mesa_clean.rename(index=str, columns={'LOCATION':'Address','SITUS_CITY':'City','ARCH1ST':'PropertySubtype','ZONING':'Zoning'})
nancheck(df_mesa_clean) ## 6836 out of 74662 records has no address
df_mesa_clean=df_mesa_clean.dropna(axis=0,how='any')   
df_mesa_clean['County']='MESA'
df_mesa_clean['CountyID']='077'
df_mesa_clean.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Shape of the dataframe: (74662, 5) 

Missing value counts:
Address             6836
City                2480
PropertySubtype    13305
Zoning             10499
Area                   0
dtype: int64 



,Address,City,PropertySubtype,Zoning,Area,County,CountyID
1,67481 BLACK BEAR DR,COLLBRAN,SMALL HOUSE,AFT,14191.199827,MESA,077
2,212 27 RD,GRAND JUNCTION,DUPLEX/TRI,RSF-4,22710.603208,MESA,077
4,803 MEASE RD,GRAND JUNCTION,RANCH,RSF-E,87463.528970,MESA,077
7,411 W CHIPETA AVE,GRAND JUNCTION,RANCH,R-8,6249.207334,MESA,077
8,378 KATHLEEN CIR,GRAND JUNCTION,RANCH,R-8,4800.677468,MESA,077


In [0]:
df_parcel=pd.concat([df_mesa_clean,df_den_clean])
df_parcel.head()

,Address,City,PropertySubtype,Zoning,Area,County,CountyID
1,67481 BLACK BEAR DR,COLLBRAN,SMALL HOUSE,AFT,14191.199827,MESA,077
2,212 27 RD,GRAND JUNCTION,DUPLEX/TRI,RSF-4,22710.603208,MESA,077
4,803 MEASE RD,GRAND JUNCTION,RANCH,RSF-E,87463.528970,MESA,077
7,411 W CHIPETA AVE,GRAND JUNCTION,RANCH,R-8,6249.207334,MESA,077
8,378 KATHLEEN CIR,GRAND JUNCTION,RANCH,R-8,4800.677468,MESA,077


In [0]:

write_checkpoint(df_den_clean,'Denver_Parcels_Clean')
write_checkpoint(df_parcel,'Mesa_Parcels_Clean')